In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import KFold



In [ ]:
data_raw = np.load(r"..\data\raw\fashion_train.npy")
data_img = [np.reshape(image[:784],(28,28)) for image in data_raw]

Below is the template matching with the averaged out templates for each category.
Results are surprisingly good, given that the model is pretty straightforward in the approach

In [3]:
# Helper function
def get_scores_main(mean_conf_matrix):
    TP = np.diag(mean_conf_matrix)
    FP = np.sum(mean_conf_matrix,axis=0)-TP
    FN = np.sum(mean_conf_matrix,axis=1)-TP
    accuracy = sum(TP)/sum(TP+FP)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F1 = 2*((precision*recall)/(precision+recall))
    return [accuracy, precision, recall, F1]
    
# get_scores(mean_conf_matrix)

In [74]:
def info(data):
    NUMBER_OF_CLASSES = 5
    kf = KFold(n_splits=5, shuffle=True)
    kf.get_n_splits(data)
    cms = []
    all_scores = []
    score_frame = pd.DataFrame()
    score_frame['label'] = data[:,-1]
    for i in range(5):
        score_frame[i] = None
    score_frame = score_frame.astype({i: 'float' for i in range(5)})
    for fold_id,(train_idx, test_idx) in enumerate(kf.split(data)):
        
        print(f"Fold {fold_id}")
        
        clothes = data[train_idx]
        mean_templates = [np.mean(clothes[clothes[:,-1]==cloth_class],axis=0) for cloth_class in range(NUMBER_OF_CLASSES)]
        validation_clothes = data[test_idx]

        test_results = pd.DataFrame(test_idx)
        results_df = pd.DataFrame()

        for class_id in range(NUMBER_OF_CLASSES):
            euclidean_distance = np.linalg.norm(validation_clothes - mean_templates[class_id][np.newaxis, :], axis=1)
            results_df[class_id] = euclidean_distance
            score_frame.loc[test_idx, class_id] = euclidean_distance

        test_results['Actual_score'] = validation_clothes[:,-1]
        test_results['Template_score'] = results_df.idxmin(axis=1)
                    
        y_test = test_results['Actual_score']
        y_pred = test_results['Template_score']

        # print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
        # print('Classification Report:\n',classification_report(y_test, y_pred))
        
        fold_scores = get_scores_main(confusion_matrix(y_test, y_pred))
        all_scores.append(fold_scores)
        cms.append(confusion_matrix(y_test, y_pred))
        # print(results_df)

    mean_conf_matrix = np.mean(cms,axis=0)
    var_conf_matrix = np.var(cms,axis=0)
    print("Mean confusion matrix\n", mean_conf_matrix)
    print()
    print("Variance of mean confusion matrix\n", var_conf_matrix)

    print("Accuracy:", round(np.mean([all_scores[i][0] for i in range(5)],axis=0),4))
    print("Std Accuracy:", round(np.std([all_scores[i][0] for i in range(5)]),4))
    print()
    print("Precision:",list(map(lambda x:round(x,4),np.mean([all_scores[i][1] for i in range(5)],axis=0))))
    print("Std Precision:",list(map(lambda x:round(x,4),np.std([all_scores[i][1] for i in range(5)],axis=0))))
    print()
    print("Recall:",list(map(lambda x:round(x,4),np.mean([all_scores[i][2] for i in range(5)],axis=0))))
    print("Std Recall:",list(map(lambda x:round(x,4),np.std([all_scores[i][2] for i in range(5)],axis=0))))
    print()
    print("F1:",list(map(lambda x:round(x,4),np.mean([all_scores[i][3] for i in range(5)],axis=0))))
    print("Std F1:",list(map(lambda x:round(x,4),np.std([all_scores[i][3] for i in range(5)],axis=0))))

In [91]:
info(data_raw)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Mean confusion matrix
 [[295.   17.8  10.6  55.   28.2]
 [  9.6 349.8   4.4  18.8   6.8]
 [  2.2   3.8 259.8   4.6 129.8]
 [ 22.8  11.6   4.4 334.6  27.6]
 [ 79.    9.  126.6  30.6 157.6]]

Variance of mean confusion matrix
 [[194.8   11.36   7.44  28.4   36.56]
 [  9.04 262.16   1.04   9.36   3.76]
 [  0.56   0.56  62.96   5.44 299.36]
 [ 31.76   7.84   2.64 132.24  60.64]
 [ 50.     4.8   75.44   2.64  37.04]]
Accuracy: 0.6984
Std Accuracy: 0.0076

Precision: [np.float64(0.722), np.float64(0.8921), np.float64(0.6404), np.float64(0.7547), np.float64(0.4524)]
Std Precision: [np.float64(0.0251), np.float64(0.0143), np.float64(0.02), np.float64(0.0148), np.float64(0.0394)]

Recall: [np.float64(0.7255), np.float64(0.8979), np.float64(0.6501), np.float64(0.8346), np.float64(0.3915)]
Std Recall: [np.float64(0.0235), np.float64(0.0147), np.float64(0.0225), np.float64(0.0212), np.float64(0.0074)]

F1: [np.float64(0.7235), np.float64(0.8949), np.float64(0.644

# Short explanation:
Per row we have 785 columns - encoded 28x28 image, plus column for a label.
Normally, in ML we scale (substract mean and subsequently divide by std) alongside ROWS! (so per column!)
But I will argue that for images it's often very often worthwhile to scale alongside columns! (so per image, per row), but not necessarily alongside rows.

Think of what scaling does when done per image.
It - kind of - sets the brightness to 0 and contrast to 1!
So, there is an assumption that the **whole** image, for example, was made in a ligher/darker place with the given camera which evenly records contrast.
I think this is a very reasonable assumption and there are no obvious doubts about that.

What would scaling per columns do?
It would that, there is an assumption that every image was taken in one session, where certain parts of the image were under different lightning and contrasting differently. This could be true, but is very much less likely.
But what is more importantly: scaling per column would assume that each pixel position has a certain distribution. In my opinion - it's either not true or useful. Each pixel position is so much heavily influenced by the contents of the image itself, that very likely instead of removing contrast/light we will bring other unnecessary artefacts from other images deterioriating quality. I'm not sure how to mathematically explain it.


In [90]:
row_scaled_data = (data_raw[:,:-1] - np.mean(data_raw[:,:-1],axis=1)[:,np.newaxis]) / np.std(data_raw[:,:-1],axis=1)[:,np.newaxis]
data_scaled = np.column_stack((row_scaled_data,data_raw[:,-1]))
info(data_scaled)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Mean confusion matrix
 [[329.8   2.    6.8  44.   24. ]
 [  8.2 352.4   7.   17.8   4. ]
 [  3.2   0.6 318.4   4.   74. ]
 [ 12.6   2.4   1.4 354.   30.6]
 [ 88.    0.8  96.6  27.8 189.6]]

Variance of mean confusion matrix
 [[388.16   0.8    2.96  40.    50.8 ]
 [  1.36  94.24   3.2    6.96   5.2 ]
 [  3.76   0.64  49.44   5.2   55.2 ]
 [ 13.84   1.44   1.04 178.8   13.84]
 [ 52.4    0.56  70.24  41.36 322.64]]
Accuracy: 0.7721
Std Accuracy: 0.0102

Precision: [np.float64(0.7459), np.float64(0.9838), np.float64(0.7402), np.float64(0.7909), np.float64(0.5874)]
Std Precision: [np.float64(0.0274), np.float64(0.0076), np.float64(0.0204), np.float64(0.0116), np.float64(0.0228)]

Recall: [np.float64(0.8109), np.float64(0.9051), np.float64(0.7957), np.float64(0.8826), np.float64(0.4699)]
Std Recall: [np.float64(0.0222), np.float64(0.0113), np.float64(0.0149), np.float64(0.0182), np.float64(0.0204)]

F1: [np.float64(0.7768), np.float64(0.9427), np.float64(0.

Here we perform both row and column scaling.
Bear in mind that scaling the whole data by columns BEFORE training should be even more accurate, because it means that validation samples already 'know' the underlying distribution of the feature!

But it's still worse!

In [94]:
row_scaled_data = (data_raw[:,:-1] - np.mean(data_raw[:,:-1],axis=1)[:,np.newaxis]) / np.std(data_raw[:,:-1],axis=1)[:,np.newaxis]
column_scaled_data = (row_scaled_data - np.mean(row_scaled_data,axis=0)) / np.std(row_scaled_data,axis=0)
data_scaled = np.column_stack((column_scaled_data,data_raw[:,-1]))
info(data_scaled)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Mean confusion matrix
 [[321.2   5.8   7.8  53.2  18.6]
 [  9.6 350.4   5.6  20.    3.8]
 [  3.4   1.8 310.6   6.6  77.8]
 [ 18.6   3.8   2.4 347.   29.2]
 [ 99.    1.  131.2  28.  143.6]]

Variance of mean confusion matrix
 [[6.2960e+01 1.0960e+01 1.7600e+00 6.3760e+01 2.5840e+01]
 [3.0400e+00 1.9704e+02 6.4000e-01 2.8400e+01 5.6000e-01]
 [3.0400e+00 4.9600e+00 6.3040e+01 5.8400e+00 4.7360e+01]
 [3.2240e+01 2.9600e+00 2.4000e-01 2.9720e+02 3.8960e+01]
 [1.1560e+02 1.6000e+00 4.6960e+01 2.4400e+01 1.6824e+02]]
Accuracy: 0.7364
Std Accuracy: 0.0077

Precision: [np.float64(0.7114), np.float64(0.9659), np.float64(0.6788), np.float64(0.7637), np.float64(0.5259)]
Std Precision: [np.float64(0.0154), np.float64(0.0106), np.float64(0.011), np.float64(0.0157), np.float64(0.0415)]

Recall: [np.float64(0.7901), np.float64(0.8995), np.float64(0.7765), np.float64(0.8658), np.float64(0.3562)]
Std Recall: [np.float64(0.0199), np.float64(0.0194), np.float64(0.0192), 

Maybe column scaling withing classes would be better?

In [98]:
row_scaled_data = (data_raw[:,:-1] - np.mean(data_raw[:,:-1],axis=1)[:,np.newaxis]) / np.std(data_raw[:,:-1],axis=1)[:,np.newaxis]

# Initialize an array to hold the scaled data
column_scaled_data = np.zeros_like(row_scaled_data)

# Perform column scaling individually for each class
for class_label in range(NUMBER_OF_CLASSES):
    class_indices = data_raw[:,-1] == class_label
    class_data = row_scaled_data[class_indices]
    class_data_scaled = (class_data - np.mean(class_data, axis=0)) / np.std(class_data, axis=0)
    column_scaled_data[class_indices] = class_data_scaled

data_scaled = np.column_stack((column_scaled_data, data_raw[:,-1]))
info(data_scaled)

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Mean confusion matrix
 [[220.4 134.4  47.8   2.6   1.4]
 [ 82.2 174.8 101.2  29.8   1.4]
 [ 21.8  64.6 162.4 133.6  17.8]
 [  5.    4.6 142.8 152.6  96. ]
 [  0.    0.   62.6 161.  179.2]]

Variance of mean confusion matrix
 [[1183.04 1778.24 2586.56   15.04    7.84]
 [1499.76 3070.96 1204.96 1027.76    7.84]
 [ 670.56 1827.44  433.04 1904.64  316.56]
 [  72.4    34.24 1288.96  883.44 3924.8 ]
 [   0.      0.   1073.04  473.2   457.36]]
Accuracy: 0.4447
Std Accuracy: 0.0426

Precision: [np.float64(0.6893), np.float64(0.4657), np.float64(0.3263), np.float64(0.3197), np.float64(0.6261)]
Std Precision: [np.float64(0.1373), np.float64(0.1241), np.float64(0.0703), np.float64(0.0568), np.float64(0.1075)]

Recall: [np.float64(0.5398), np.float64(0.4469), np.float64(0.4055), np.float64(0.3835), np.float64(0.4459)]
Std Recall: [np.float64(0.0636), np.float64(0.1316), np.float64(0.0459), np.float64(0.0813), np.float64(0.0584)]

F1: [np.float64(0.5998), np.float

In [54]:
score_frame.to_csv(r"..\data\interim\template_matching.csv", index=False)

Below is the template matching by taking the mean and dividing it over the variance, measuring the distance from that. Either I have done it in a not intended way or it is just not a reliable way of predicting values